In [1]:
import pickle 
import numpy as np

In [2]:
with open('train_qa.txt','rb') as f :
    train_data = pickle.load(f)

In [3]:
with open('test_qa.txt','rb') as f :
    test_data = pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(train_data)

10000

In [7]:
len(test_data)

1000

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [9]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [10]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [11]:
' '.join(train_data[0][2])

'n o'

In [12]:
all_data = test_data + train_data

In [13]:
len(all_data)

11000

In [14]:
vocab = set()

for story,question,answer in all_data :
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [15]:
vocab.add('no')

In [16]:
vocab.add('yes')

In [17]:
vocab_len = len(vocab) + 1

In [18]:
vocab_len

38

In [19]:
all_story_lens=[len(data[0]) for data in all_data]

In [20]:
max(all_story_lens)

156

In [21]:
max_story_len =max(all_story_lens)

In [22]:
max_question_len = max([len(data[1]) for data in all_data])

In [23]:
max_question_len

6

In [24]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [25]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [26]:
tokenizer.word_index

{'took': 1,
 'hallway': 2,
 'left': 3,
 'bathroom': 4,
 'back': 5,
 'bedroom': 6,
 'down': 7,
 'sandra': 8,
 'up': 9,
 '.': 10,
 'there': 11,
 'discarded': 12,
 'dropped': 13,
 'put': 14,
 'yes': 15,
 '?': 16,
 'in': 17,
 'kitchen': 18,
 'is': 19,
 'to': 20,
 'got': 21,
 'mary': 22,
 'travelled': 23,
 'garden': 24,
 'apple': 25,
 'daniel': 26,
 'journeyed': 27,
 'no': 28,
 'moved': 29,
 'milk': 30,
 'went': 31,
 'the': 32,
 'john': 33,
 'football': 34,
 'office': 35,
 'picked': 36,
 'grabbed': 37}

In [27]:
tokenizer.word_index['?']

16

In [28]:
train_story_text = []
train_question_text = []
train_answers = []

In [29]:
for story, question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [30]:
train_story_text[0]

['Mary',
 'moved',
 'to',
 'the',
 'bathroom',
 '.',
 'Sandra',
 'journeyed',
 'to',
 'the',
 'bedroom',
 '.']

In [31]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [32]:
len(train_story_seq)

10000

In [33]:
train_story_seq[0]

[22, 29, 20, 32, 4, 10, 8, 27, 20, 32, 6, 10]

In [34]:
def vectorize_stories(data,word_index = tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    #stories = x
    X = []
    #questions Xq
    Xq = []
    #Y corrrect answerss
    Y =[]
    
    for story,query,answer in data :
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        
        y = np.zeros(len(word_index)+1)
        y[word_index[answer]] =1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)    
    
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [35]:
inputs_train,queries_train,answers_train = vectorize_stories(train_data)

In [36]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [37]:
inputs_test

array([[ 0,  0,  0, ..., 32,  6, 10],
       [ 0,  0,  0, ..., 32, 24, 10],
       [ 0,  0,  0, ..., 32, 24, 10],
       ...,
       [ 0,  0,  0, ..., 32, 25, 10],
       [ 0,  0,  0, ..., 32, 24, 10],
       [ 0,  0,  0, ..., 25, 11, 10]], dtype=int32)

In [38]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
tokenizer.word_index['yes']

15

In [40]:
tokenizer.word_index['no']

28

In [41]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0., 497.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0., 503.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [42]:
from keras.models import Sequential,Model

In [43]:
from keras.layers.embeddings import Embedding

In [44]:
from keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM

In [45]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [46]:
vocab_size = len(vocab) +1

In [47]:
#INPUT ENCODER M

input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [48]:
#INPUT ENCODER c

input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))


In [49]:
#INPUT ENCODER c

question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))



In [50]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [51]:
match = dot([input_encoded_m,question_encoded],axes=(2,2))
match = Activation('softmax') (match)

In [52]:
response = add([match,input_encoded_c])
response = Permute((2,1))(response)

In [53]:
answer = concatenate([response,question_encoded])

In [54]:
answer

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 6, 220) dtype=float32>

In [55]:
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)

In [56]:
answer = Activation('softmax')(answer)

In [57]:
model = Model([input_sequence,question],answer)

In [58]:
model.compile(optimizer='rmsprop',loss= 'categorical_crossentropy',metrics=['accuracy'])

In [59]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 156)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [60]:
history = model.fit([inputs_train, queries_train], 
                    answers_train,
                    batch_size=32,
                    epochs=3,
                    validation_data=([inputs_test, queries_test], answers_test))

IndentationError: unexpected indent (<ipython-input-60-5df551713a64>, line 2)

In [61]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [62]:
model.load_weights('chatbot_10.h5')

In [ ]:
pred_results = model.predict(([inputs_test, queries_test]))